In [1]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geomloss import SamplesLoss
import torch

[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode


In [2]:
gan_swat_network = pd.read_csv("gan_lstm_swat_network.csv", sep=",").to_numpy()
gan_swat_sensor = pd.read_csv("gan_lstm_swat_sensor.csv", sep=",").to_numpy()
gan_wadi = pd.read_csv("gan_lstm_wadi.csv", sep=",").to_numpy()
gan_wustl = pd.read_csv("gan_lstm_wustl.csv", sep=",").to_numpy()

vae_swat_network = pd.read_csv("vae_swat_network.csv", sep=",").to_numpy()
vae_swat_sensor = pd.read_csv("vae_swat_sensor.csv", sep=",").to_numpy()
vae_wadi = pd.read_csv("vae_wadi_sensor.csv", sep=",").to_numpy()
vae_wustl = pd.read_csv("vae_wustl_network.csv", sep=",").to_numpy()

vae_gan_swat_network = pd.read_csv("vae_gan_swat_network.csv", sep=",").to_numpy()
vae_gan_swat_sensor = pd.read_csv("vae_gan_swat_sensor.csv", sep=",").to_numpy()
vae_gan_wadi = pd.read_csv("vae_gan_wadi_sensor.csv", sep=",").to_numpy()
vae_gan_wustl = pd.read_csv("vae_gan_wustl_network.csv", sep=",").to_numpy()

gan_softmax_swat_network = pd.read_csv("gan_softmax_swat_network_50.csv", sep=",").to_numpy()
gan_softmax_swat_sensor = pd.read_csv("gan_softmax_swat_sensor_50.csv", sep=",").to_numpy()
gan_softmax_wadi = pd.read_csv("gan_softmax_wadi_50.csv", sep=",").to_numpy()
gan_softmax_wustl = pd.read_csv("gan_softmax_wustl_50.csv", sep=",").to_numpy()

In [3]:
swat = pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/swat_processed.csv", sep=",").to_numpy()[:, 1:]
# swap sport to front
swat[:, [0, 1]] = swat[:, [1, 0]]
# # swap dport to after sport
swat[:, [1, 2]] = swat[:, [2, 1]]
# # swap protocols to after dport
swat[:, [2, 14]] = swat[:, [14, 2]]

wadi = pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/wadi_attacks.csv", sep=",").to_numpy()[:, 1:]
swat_sensor = pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/swat_sensor.csv", sep=",").to_numpy()[:, 1:]
wustl = pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/wustl_attacks.csv", sep=",").to_numpy()[:, 1:]

In [4]:
loss = SamplesLoss("sinkhorn", blur=0.05)

In [5]:
def wasserstein(real, fake):
    loss = SamplesLoss("sinkhorn", blur=0.05)

    num_features = real.shape[1]
    real = torch.from_numpy(real[: fake.shape[0]])
    fake = torch.from_numpy(fake)

    return loss(real, fake)

## GAN

In [6]:
wasserstein(swat, gan_swat_network[:, 1:])

tensor(3.2370e+16, dtype=torch.float64)

In [7]:
wasserstein(swat_sensor, gan_swat_sensor)

tensor(1177256.3420, dtype=torch.float64)

In [8]:
wasserstein(wadi, gan_wadi)

tensor(37967923.8757, dtype=torch.float64)

In [9]:
wasserstein(wustl, gan_wustl)

tensor(1.2848e+18, dtype=torch.float64)

## VAE

In [10]:
wasserstein(swat, vae_swat_network[:5000, 1:])

tensor(3.2370e+16, dtype=torch.float64)

In [13]:
wasserstein(swat_sensor, vae_swat_sensor)

tensor(1160966.8480, dtype=torch.float64)

In [ ]:
wasserstein(wadi, vae_wadi)

tensor(38058719.4092, dtype=torch.float64)

In [14]:
wasserstein(wustl, vae_wustl[:5000])

tensor(1.2832e+18, dtype=torch.float64)

## VAE+GAN

In [15]:
wasserstein(swat, vae_gan_swat_network[:, 1:])

tensor(3.1993e+16, dtype=torch.float64)

In [16]:
wasserstein(swat_sensor, vae_gan_swat_sensor.reshape(2640, 28))

tensor(1162073.4082, dtype=torch.float64)

In [17]:
wasserstein(wadi, vae_gan_wadi.reshape(1996, 69))

tensor(38063814.5702, dtype=torch.float64)

In [18]:
wasserstein(wustl, vae_gan_wustl.reshape(17404, 27)[:5000])

tensor(1.2832e+18, dtype=torch.float64)

## GAN Softmax

In [ ]:
wasserstein(swat, gan_softmax_swat_network)

tensor(3.2370e+16, dtype=torch.float64)

In [ ]:
wasserstein(swat_sensor, gan_softmax_swat_sensor)

tensor(1182450.7273, dtype=torch.float64)

In [ ]:
wasserstein(wadi, gan_softmax_wadi)

tensor(37797590.5259, dtype=torch.float64)

In [ ]:
wasserstein(wustl, gan_softmax_wustl)

tensor(1.2832e+18, dtype=torch.float64)